In [ ]:
import json
#import seaborn as sns 
#import matplotlib.pyplot as plt

In [ ]:
mainfile = "result_2023_6_8.json"
smallfile = "results_2023_5_31.json"

In [ ]:
maindata = []
with open(mainfile, 'r') as f:
  for entry in f:
      oneEntry = json.loads(entry)
      maindata.append(oneEntry)

In [ ]:
filtered_data = []
for d in maindata: 
  if d.get('report') != None: #not empty
    if d.get('report').get('user_id') != None: 
      filtered_data.append(d)

In [ ]:
type(maindata)

In [ ]:
print(len(maindata),
len(filtered_data))

In [ ]:
empty_data = []
for d in maindata: 
    if d.get('report') != None: 
        if d.get('report').get('user_id') == None:
            empty_data.append(d)

In [ ]:
len(empty_data)

In [ ]:
len(empty_data)/len(maindata)

In [ ]:
len(filtered_data)

In [ ]:
filtered_data[2]

In [ ]:
'''example'''
print(filtered_data[0]['report']['tracker_info']['tracker']['owner']['url'])

In [ ]:
#get all urls in a list
urls = []
for d in filtered_data: 
    rep = d['report']
    if rep.get('tracker_info') != None: #some reports do not have tracker info
        if rep['tracker_info'].get('tracker').get('owner').get('url') != None: #all reports with tracker info, have the following
            urls.append(d['report']['tracker_info']['tracker']['owner']['url'])
print(len(urls))

In [ ]:
print(urls[0:10])

In [ ]:
len(urls)/len(filtered_data)

In [ ]:
print(len(set(urls)))

In [ ]:
def produce_count_dict(list):
    count_d = {}
    for i in list: 
        if i in count_d: 
            a = count_d.get(i)
            count_d[i] = a + 1
        else: 
            count_d[i] = 1
            
    return dict(sorted(count_d.items(), key=lambda x:x[1], reverse = True))

In [ ]:
type(produce_count_dict(urls))

In [ ]:
#counting how many times a url appears in the reports, and sorting in order
sorted_count_urls = produce_count_dict(urls)
sorted_count_urls

interesting websites from a bird eye view: 
- a few web add related websites (eg. tradedesk, smartadserver, adyoulike, adform, gumgum)
- paypal has 13 findings --> google, amazon also have credit card details

also, interesting is to look at the top ones with obscure websites eg. 
 ('http://pubmatic.com', 896), --> media company "digital marketing" 
 ('http://sharethrough.com', 301), --> "human centric programmatic advertising" 
 
I guess this makes sense that these are the ones picked up, as they do collect user data

## What the data is

In [ ]:
example_report = filtered_data[3]['report']
example_report

In [ ]:
#what is leak_url and what is url_leak_type
leak_type = []

for i in filtered_data: 
    if i.get('report') != None: 
        leak_type.append(i['report']['url_leak_type'])
        
print(len(leak_type))

In [ ]:
nonempty_leak_type = []
for i in leak_type: 
    if i != None: 
        nonempty_leak_type.append(i)

In [ ]:
print(len(nonempty_leak_type), nonempty_leak_type[1], type(nonempty_leak_type[1]))

In [ ]:
def return_unique(l: list): 
    '''
    A function that does the same thing as a set(), but for unhashable types
    '''
    a = set()
    
    [[a.add(field) for field in item] for item in l]

    return a

In [ ]:
return_unique(nonempty_leak_type)

In [ ]:
test_1 = list(set(tuple(sorted(sub)) for sub in nonempty_leak_type))
test_1

In [ ]:
def produce_count_dict(l):
    d_keys = list(set(tuple(sorted(sub)) for sub in l))
    count_d = {field: None for field in d_keys}
    for i in l: 
        if sorted(i) in count_d: 
            a = count_d.get(j)
            count_d[j] = a + 1
        else: 
            count_d[j] = 1
    #sorting data
    sorted_count = sorted(count_d.items(), key=lambda x:x[1], reverse = True)

    return count_d

In [ ]:
filtered_data[3]['report']

things to do next
- script to output a markdown/text file outputting the da
- potential graphs
- understanding trends (time series)
- other types of data to look at 
- especially under fb/google/etc

- create a started page for extension 
- then group by User

also, explain what one entry is
- does the order of the field counted matter? 
- should we open these tuples and sort by field itself, or as a packaged tuple

## connecting field types with company

In [ ]:
final_count = {}

for i in filtered_data: 
    if i['report'].get('tracker_info') != None: #some reports do not have tracker info
        if i['report']['tracker_info'].get('tracker').get('owner').get('url') != None: #all reports w/ tracker info has the owner/url link
            #getting urls (which will be the keys)
            url = i['report']['tracker_info']['tracker']['owner']['url']
            #getting leak types (which will be the keys for the nested dict for each url)
            if i['report']['url_leak_type'] is None: 
                leak_type = "None"
            else: 
                leak_type = i['report']['url_leak_type']
                leak_type = tuple(sorted(leak_type)) #ADDING THE SORTED REMOVES FIELD DUPLICATES
                            
            if url in final_count: 
                #dic.get('b') == None: <-- another way to find 
                if leak_type in final_count[url]: 
                    final_count[url][leak_type] += 1
                else: 
                    final_count[url][leak_type] = 1
            else: 
                final_count[url] = {} #results in empty field dicts for some urls

In [ ]:
final_count

ideal output: <br>

{ 'website.com': {<br>
&emsp;&emsp;    field_name_1: 23 <br>
&emsp;&emsp;    field_name_2: 34 <br>
&emsp;&emsp;    field_name_3: 12<br>
&emsp;&emsp;        }<br>
   'website2.com': {<br>
     &emsp;&emsp;  field_name_2: 124<br>
     &emsp;&emsp;  field_name_4: 81<br>
 &emsp;&emsp;    } <br>
}

In [ ]:
a = list(sorted_count_urls.keys())
a[0:14]

Questions: 
- why are we picking up so many None types even after passing data thru filters?
- We should clean out the companies that have empty fields and 
        - 1) make a proportion measurement of empty to non empty
        - 2) add a filter hashtag to the companies by maybe doing easy webcrawl/nlp search and assigning if company is #media, #tech ... and then maybe categorize level of harm that the leaks are
- what prof. marshini said, about cateogiring by user instead of company

In [ ]:
#matplotlib, ##seaport, pyplot 